### First we import packages, set the season_id to 2024

In [2]:
import requests
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
season_id = 2024

### Then string format the request URL so season can change -- request url comes from ESPN API

In [3]:
url = "https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/{}/segments/0/leaguedefaults/3?view=kona_player_info".format(season_id)
# historical call
url_22 = "https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/2022/segments/0/leaguedefaults/3?view=kona_player_info"

In [4]:
# create a cookies variable to tell ESPN its you making URL call
espn_cookies = {"swid" : "{27EEB99A-12A0-4549-9D00-79FE69FFF90E}", 
               "espn_s2" : "AECz9ORRiySSOCBHZaLyYGJNnl5weoG3CExU9x%2FGA75Yf%2B6DNxOB%2BBRl8FXEtJn7lJtE6ya4FMQ%2BJLZvkd9uiR5I73e7bQ1Nq0OWnHRMZMkLMc1W1FNTcIv%2Bu3SuigNnODtR9%2Fv0e8l%2BFJnAETUWuf6hAtTgBG5j%2Fobg1YxRCcicIO9O17jWzmuEhQrTKuy4YTRt25boSxNKsj46tq8K0glsOjuq2tUpLqPzJjBCm%2FAkbiQV4gSE44yiUG6TUSFfrL8tATIkYpjerN2y4n%2BT9lHabsvdyi5Bnxs43jSa%2Flv4NzJUfR7EoXmDe0YOojzyPno%3D"}


In [5]:
# headers needed as parameter for GET request, filter changed to none 
headers = {
 'Connection': 'keep-alive',
 'Accept': 'application/json, text/plain, */*',
 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
'X-fantasy-filter': '{"players":{"filterSlotIds":{"value":[0,2,23,4,6]},"sortAppliedStatTotal":{"sortAsc":false,"sortPriority":2,"value":"002023"},"sortAppliedStatTotalForScoringPeriodId":null,"sortStatId":null,"sortStatIdForScoringPeriodId":null,"sortPercOwned":{"sortPriority":3,"sortAsc":false},"limit":null,"filterRanksForSlotIds":{"value":[0,2,4,6,17,16,8,9,10,12,13,24,11,14,15]},"filterStatsForTopScoringPeriodIds":{"value":2,"additionalValue":["002024","102024","002023","022024"]}}}',
 'X-fantasy-platform': 'kona-PROD-18c32c6242c08d37606f52ca971782183e620e50',
 'X-fantasy-source': 'kona'
}

In [6]:
r = requests.get(url, headers=headers, cookies=espn_cookies) 
espn_raw_data = r.json()

# create DF with desired columns
espn_data_detail = espn_raw_data['players']
df = pd.DataFrame(espn_data_detail)
players = df[['player', 'ratings', 'id']]

In [7]:
#experiment for previous years
url_22 = "https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/2022/segments/0/leaguedefaults/3?view=kona_player_info"
r_22 = requests.get(url_22, headers=headers, cookies = espn_cookies)
espn_data_22 = r_22.json()

espn_data_22 = espn_data_22['players']
df = pd.DataFrame(espn_data_22)
players_22 = df[['player', 'ratings', 'id']]

In [9]:
players_22['player'][1]

{'active': True,
 'defaultPositionId': 1,
 'droppable': False,
 'eligibleSlots': [0, 7, 20, 21],
 'firstName': 'Josh',
 'fullName': 'Josh Allen',
 'id': 3918298,
 'injured': False,
 'injuryStatus': 'ACTIVE',
 'jersey': '17',
 'lastName': 'Allen',
 'ownership': {'activityLevel': None,
  'auctionValueAverage': 17.59223300970874,
  'auctionValueAverageChange': 0.0,
  'averageDraftPosition': 16.630930656934307,
  'averageDraftPositionPercentChange': 0.0,
  'date': None,
  'leagueType': 0,
  'percentChange': 0.0048428967332938555,
  'percentOwned': 99.95465331926991,
  'percentStarted': 95.44198017171752},
 'proTeamId': 2,
 'stats': [{'appliedTotal': 21.759999999999998,
   'externalId': '401437949',
   'id': '01401437949',
   'proTeamId': 2,
   'scoringPeriodId': 18,
   'seasonId': 2022,
   'statSourceId': 0,
   'statSplitTypeId': 1,
   'stats': {'0': 31.0,
    '1': 19.0,
    '2': 12.0,
    '3': 254.0,
    '4': 3.0,
    '5': 50.0,
    '6': 25.0,
    '7': 12.0,
    '8': 10.0,
    '9': 5.0,
 

In [99]:
# some dictionaries that will help map variables later
teams = {0:'FA', 1:'Atlanta Falcons', 2:'Buffalo Bills', 3:'Chicago Bears', 4:'Cincinnati Bengals', 5:'Cleveland Browns', 
         6:'Dallas Cowboys', 7:'Denver Broncos', 8:'Detroit Lions', 9:'Green Bay Packers',10:'Tennessee Titans',
         11:'Indianapolis Colts', 12:'Kansas City Chiefs', 13:'Las Vegas Raiders', 14:'Los Angeles Rams', 15:'Miami Dolphins', 
         16:'Minnesota Vikings', 17:'New England Patriots', 18:'New Orleans Saints', 19:'New York Giants', 20:'New York Jets',
         21:'Philadelphia Eagles', 22:'Arizona Cardinals', 23:'Pittsburgh Steelers', 24:'Los Angeles Chargers', 25:'San Francisco 49ers',
         26:'Seattle Seahawks', 27:'Tampa Bay Buccaneers', 28:'Washington Commanders', 29:'Carolina Panthers', 30:'Jacksonville Jaguars', 31:'', 32:'', 33:'Baltimore Ravens', 34:'Houston Texans'}

positions = {1:'QB', 2:'RB', 3:'WR', 4:'TE', 5:'K', 16:'D/ST'}

espn_stat_key = {0:'attempts', 1:'completions', 3:'pass_yds', 4:'pass_td', 20: 'interceptions', 23:'carries', 24:'rush_yds', 25:'rush_td', 42:'rec_yds', 43:'rec_td', 53:'receptions'}


In [114]:
#The following set of functions might be unnecessarily complicated but its because the architecture of ESPN's API has so many nested dictionaries
# for example: 
#players['player'][0]

In [128]:
# need to flatten 'ratings' dictionary (this is kind of unnecessary? i think i could have just used a .get() on the dictionary)
def extract_inner_dict(row, col_name):
    outer_dict = row[col_name]
    # Get the inner dictionary by accessing the first key of the outer dictionary
    inner_dict = next(iter(outer_dict.values()))
    return inner_dict

# Apply the function to the DataFrame column
players['ratings'] = players.apply(lambda row: extract_inner_dict(row, 'ratings'), axis=1)

In [129]:

# Function to extract player details with default keys for missing values
def extract_player(dictionary):
    name = dictionary.get('fullName', 'Unknown')
    status = dictionary.get('injuryStatus', 'Unknown')
    pos_id = dictionary.get('defaultPositionId', 'Unknown')
    team_id = dictionary.get('proTeamId', 'Unknown')
    return pd.Series([name, status, pos_id, team_id])

# Function to extract ranking details
def extract_rank_23(dictionary):
    rank_23 = dictionary.get('totalRanking')
    positional_rank_23 = dictionary.get('positionalRanking')
    total_points_23 = dictionary.get('totalRating')
    return pd.Series([rank_23, positional_rank_23, total_points_23])

# Apply the functions and assign the results to new columns
players[['name', 'status', 'pos_id', 'team_id']] = players['player'].apply(extract_player)
players[['rank_23', 'positional_rank_23', 'total_points_23']] = players['ratings'].apply(extract_rank_23)

players.head()

,player,ratings,id,name,status,pos_id,team_id,rank_23,positional_rank_23,total_points_23
0,"{'active': True, 'defaultPositionId': 3, 'draf...","{'positionalRanking': 1, 'totalRanking': 1, 't...",4241389,CeeDee Lamb,ACTIVE,3,6,1.0,1.0,403.20
1,"{'active': True, 'defaultPositionId': 1, 'draf...","{'positionalRanking': 1, 'totalRanking': 2, 't...",3918298,Josh Allen,ACTIVE,1,2,2.0,1.0,392.64
2,"{'active': True, 'defaultPositionId': 2, 'draf...","{'positionalRanking': 1, 'totalRanking': 3, 't...",3117251,Christian McCaffrey,ACTIVE,2,25,3.0,1.0,391.30
3,"{'active': True, 'defaultPositionId': 3, 'draf...","{'positionalRanking': 2, 'totalRanking': 4, 't...",3116406,Tyreek Hill,ACTIVE,3,15,4.0,2.0,376.40
4,"{'active': True, 'defaultPositionId': 1, 'draf...","{'positionalRanking': 2, 'totalRanking': 5, 't...",4040715,Jalen Hurts,ACTIVE,1,21,5.0,2.0,356.82


In [130]:
#Create avg point
players['avg_points_23'] = players['total_points_23']/17

In [131]:
# create team and position dictionaries to map ids
players['team'] = players['team_id'].map(teams) 
players['position'] = players['pos_id'].map(positions)

In [132]:
# Reordering columns
players = players[['id', 'name', 'status', 'rank_23', 'positional_rank_23', 'total_points_23', 'avg_points_23', 'team', 'position', 'pos_id','team_id', 'player', 'ratings']]

In [133]:
# changing rank & pos_rank to integers, rounding off points 
players[['rank_23', 'positional_rank_23']] = players[['rank_23', 'positional_rank_23']].astype(int)
players = players.round({'total_points_23':1, 'avg_points_23':2})

In [134]:
players.head()

,id,name,status,rank_23,positional_rank_23,total_points_23,avg_points_23,team,position,pos_id,team_id,player,ratings
0,4241389,CeeDee Lamb,ACTIVE,1,1,403.2,23.72,Dallas Cowboys,WR,3,6,"{'active': True, 'defaultPositionId': 3, 'draf...","{'positionalRanking': 1, 'totalRanking': 1, 't..."
1,3918298,Josh Allen,ACTIVE,2,1,392.6,23.10,Buffalo Bills,QB,1,2,"{'active': True, 'defaultPositionId': 1, 'draf...","{'positionalRanking': 1, 'totalRanking': 2, 't..."
2,3117251,Christian McCaffrey,ACTIVE,3,1,391.3,23.02,San Francisco 49ers,RB,2,25,"{'active': True, 'defaultPositionId': 2, 'draf...","{'positionalRanking': 1, 'totalRanking': 3, 't..."
3,3116406,Tyreek Hill,ACTIVE,4,2,376.4,22.14,Miami Dolphins,WR,3,15,"{'active': True, 'defaultPositionId': 3, 'draf...","{'positionalRanking': 2, 'totalRanking': 4, 't..."
4,4040715,Jalen Hurts,ACTIVE,5,2,356.8,20.99,Philadelphia Eagles,QB,1,21,"{'active': True, 'defaultPositionId': 1, 'draf...","{'positionalRanking': 2, 'totalRanking': 5, 't..."


In [135]:
players[players['name']=='Travis Kelce']

,id,name,status,rank_23,positional_rank_23,total_points_23,avg_points_23,team,position,pos_id,team_id,player,ratings
58,15847,Travis Kelce,ACTIVE,59,3,219.4,12.91,Kansas City Chiefs,TE,4,12,"{'active': True, 'defaultPositionId': 4, 'draf...","{'positionalRanking': 3, 'totalRanking': 59, '..."


### now I should be able to import historical data and join it by the ID

In [109]:
# package i thought might be helpful but wasn't
# from sportsipy.nfl.boxscore import Boxscore

In [138]:
players_copy = players.copy()

### below is just some experimenting with dictionaries and series i didn't know about

In [139]:
#players_copy['player'][0].get('stats', 'none')[3].get('stats', 'none').get('1', 0)

In [140]:
#stat_dictionary = players_copy['player'][0].get('stats')[3].get('stats')

#keys_to_extract = ['0', '1', '3', '4', '20', '23', '24', '25', '42', '43', '53']
# initializing an empty dictionary for extracted keys
#clean_stat_dict = {}
# extracting keys using a for loop and conditional statement
#for key, value in stat_dictionary.items():
    #if key in keys_to_extract:
        #clean_stat_dict[key] = value
 
#ser = pd.Series(data=clean_stat_dict)
#ser


In [141]:
# this function goes into the player dictionary and extracts the mess of nested dictionaries to get the actual stats that I want
def extract_stats_2023(dictionary):
    try:
        stats_list = dictionary.get('stats', [])

        # Initialize variables with default values
        attempts_23 = completions_23 = pass_yds_23 = pass_td_23 = interceptions_23 = carries_23 = rush_yds_23 = rush_td_23 = rec_yds_23 = rec_td_23 = receptions_23 = 0

        # Loop through stats_list to find the correct set of stats
        for stats_dict in stats_list:
            if stats_dict.get('id') == "002023":  # Assuming you want the statSplitTypeId 0
                attempts_23 = stats_dict.get('stats', {}).get('0', 0)
                completions_23 = stats_dict.get('stats', {}).get('1', 0)
                pass_yds_23 = stats_dict.get('stats', {}).get('3', 0)
                pass_td_23 = stats_dict.get('stats', {}).get('4', 0)
                interceptions_23 = stats_dict.get('stats', {}).get('20', 0)
                carries_23 = stats_dict.get('stats', {}).get('23', 0)
                rush_yds_23 = stats_dict.get('stats', {}).get('24', 0)
                rush_td_23 = stats_dict.get('stats', {}).get('25', 0)
                rec_yds_23 = stats_dict.get('stats', {}).get('42', 0)
                rec_td_23 = stats_dict.get('stats', {}).get('43', 0)
                receptions_23 = stats_dict.get('stats', {}).get('53', 0)
                break  # Stop after finding the first match

        return pd.Series([attempts_23, completions_23, pass_yds_23, pass_td_23, interceptions_23, carries_23, rush_yds_23, rush_td_23, rec_yds_23, rec_td_23, receptions_23])

    except Exception as e:
        print(f"Error processing dictionary: {dictionary}")
        print(f"Exception: {e}")
        return pd.Series([0] * 10)  # Return default values if an error occurs (thanks chatGPT)

#players_copy['player'] contains the dictionaries you want to process
players_copy[['attempts_23', 'completions_23', 'pass_yds_23', 'pass_td_23', 'interceptions_23', 
              'carries_23', 'rush_yds_23', 'rush_td_23', 'rec_yds_23', 'rec_td_23', 'receptions_23']] = players_copy['player'].apply(extract_stats_2023)
players_copy

,id,name,status,rank_23,positional_rank_23,total_points_23,avg_points_23,team,position,pos_id,...,completions_23,pass_yds_23,pass_td_23,interceptions_23,carries_23,rush_yds_23,rush_td_23,rec_yds_23,rec_td_23,receptions_23
0,4241389,CeeDee Lamb,ACTIVE,1,1,403.2,23.72,Dallas Cowboys,WR,3,...,0.0,0.0,0.0,0.0,14.0,113.0,2.0,1749.0,12.0,135.0
1,3918298,Josh Allen,ACTIVE,2,1,392.6,23.10,Buffalo Bills,QB,1,...,385.0,4306.0,29.0,18.0,111.0,524.0,15.0,0.0,0.0,0.0
2,3117251,Christian McCaffrey,ACTIVE,3,1,391.3,23.02,San Francisco 49ers,RB,2,...,0.0,0.0,0.0,0.0,272.0,1459.0,14.0,564.0,7.0,67.0
3,3116406,Tyreek Hill,ACTIVE,4,2,376.4,22.14,Miami Dolphins,WR,3,...,0.0,0.0,0.0,0.0,6.0,15.0,0.0,1799.0,13.0,119.0
4,4040715,Jalen Hurts,ACTIVE,5,2,356.8,20.99,Philadelphia Eagles,QB,1,...,352.0,3858.0,23.0,15.0,157.0,605.0,15.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,4361653,Devin Leary,ACTIVE,0,0,0.0,0.00,Baltimore Ravens,QB,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
945,4361112,Devin Culp,ACTIVE,0,0,0.0,0.00,Tampa Bay Buccaneers,TE,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
946,4569559,Devaughn Vele,ACTIVE,0,0,0.0,0.00,Denver Broncos,WR,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
947,4875912,Tejhaun Palmer,ACTIVE,0,0,0.0,0.00,Arizona Cardinals,WR,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Next... bring in 2024 projections same as for 2023 stats --- ID changes in function but nothing else

In [142]:
def extract_proj_24(dictionary):
    try:
        stats_list = dictionary.get('stats', [])

        # Initialize variables with default values
        attempts_proj_24 = completions_proj_24 = pass_yds_proj_24 = pass_td_proj_24 = interceptions_proj_24 = carries_proj_24 = rush_yds_proj_24 = rush_td_proj_24 = rec_yds_proj_24 = rec_td_proj_24 = receptions_proj_24 = 0

        # Loop through stats_list to find the correct set of stats
        for stats_dict in stats_list:
            if stats_dict.get('id') == "102024":  # Assuming you want the statSplitTypeId 0
                attempts_proj_24 = stats_dict.get('stats', {}).get('0', 0)
                completions_proj_24 = stats_dict.get('stats', {}).get('1', 0)
                pass_yds_proj_24 = stats_dict.get('stats', {}).get('3', 0)
                pass_td_proj_24 = stats_dict.get('stats', {}).get('4', 0)
                interceptions_proj_24 = stats_dict.get('stats', {}).get('20', 0)
                carries_proj_24 = stats_dict.get('stats', {}).get('23', 0)
                rush_yds_proj_24 = stats_dict.get('stats', {}).get('24', 0)
                rush_td_proj_24 = stats_dict.get('stats', {}).get('25', 0)
                rec_yds_proj_24 = stats_dict.get('stats', {}).get('42', 0)
                rec_td_proj_24 = stats_dict.get('stats', {}).get('43', 0)
                receptions_proj_24 = stats_dict.get('stats', {}).get('53', 0)
                break  # Stop after finding the first match

        return pd.Series([attempts_proj_24, completions_proj_24, pass_yds_proj_24, pass_td_proj_24, interceptions_proj_24, carries_proj_24, rush_yds_proj_24, rush_td_proj_24, rec_yds_proj_24, rec_td_proj_24, receptions_proj_24])

    except Exception as e:
        print(f"Error processing dictionary: {dictionary}")
        print(f"Exception: {e}")
        return pd.Series([0] * 10)  # Return default values if an error occurs (thanks chatGPT)

#players_copy['player'] contains the dictionaries you want to process
players_copy[['attempts_proj_24', 'completions_proj_24', 'pass_yds_proj_24', 'pass_td_proj_24', 'interceptions_proj_24', 
              'carries_proj_24', 'rush_yds_proj_24', 'rush_td_proj_24', 'rec_yds_proj_24', 'rec_td_proj_24', 'receptions_proj_24']] = players_copy['player'].apply(extract_proj_24)
players_copy

,id,name,status,rank_23,positional_rank_23,total_points_23,avg_points_23,team,position,pos_id,...,completions_proj_24,pass_yds_proj_24,pass_td_proj_24,interceptions_proj_24,carries_proj_24,rush_yds_proj_24,rush_td_proj_24,rec_yds_proj_24,rec_td_proj_24,receptions_proj_24
0,4241389,CeeDee Lamb,ACTIVE,1,1,403.2,23.72,Dallas Cowboys,WR,3,...,0.000000,0.000000,0.000000,0.000000,12.586774,85.778283,0.783793,1462.949998,8.856855,116.865698
1,3918298,Josh Allen,ACTIVE,2,1,392.6,23.10,Buffalo Bills,QB,1,...,341.283481,3715.846647,23.077842,12.369920,94.695354,498.032762,8.545350,0.000000,0.000000,0.000000
2,3117251,Christian McCaffrey,ACTIVE,3,1,391.3,23.02,San Francisco 49ers,RB,2,...,0.000000,0.000000,0.000000,0.000000,262.906740,1242.305467,12.047149,526.774494,3.842288,67.130255
3,3116406,Tyreek Hill,ACTIVE,4,2,376.4,22.14,Miami Dolphins,WR,3,...,0.000000,0.000000,0.000000,0.000000,4.099647,24.787526,0.261915,1437.745826,7.513137,106.342361
4,4040715,Jalen Hurts,ACTIVE,5,2,356.8,20.99,Philadelphia Eagles,QB,1,...,320.052976,3626.080713,20.997223,11.216677,124.534329,536.613050,9.982034,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,4361653,Devin Leary,ACTIVE,0,0,0.0,0.00,Baltimore Ravens,QB,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
945,4361112,Devin Culp,ACTIVE,0,0,0.0,0.00,Tampa Bay Buccaneers,TE,4,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.006215,0.137946,1.895157
946,4569559,Devaughn Vele,ACTIVE,0,0,0.0,0.00,Denver Broncos,WR,3,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
947,4875912,Tejhaun Palmer,ACTIVE,0,0,0.0,0.00,Arizona Cardinals,WR,3,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [143]:
def player_stats(name): 
    return players_copy[players_copy['name']==name]

In [144]:
player_stats('CeeDee Lamb')

,id,name,status,rank_23,positional_rank_23,total_points_23,avg_points_23,team,position,pos_id,...,completions_proj_24,pass_yds_proj_24,pass_td_proj_24,interceptions_proj_24,carries_proj_24,rush_yds_proj_24,rush_td_proj_24,rec_yds_proj_24,rec_td_proj_24,receptions_proj_24
0,4241389,CeeDee Lamb,ACTIVE,1,1,403.2,23.72,Dallas Cowboys,WR,3,...,0.0,0.0,0.0,0.0,12.586774,85.778283,0.783793,1462.949998,8.856855,116.865698


### RBS

In [ ]:
url_rbs = "https://site.web.api.espn.com/apis/common/v3/sports/football/nfl/statistics/byathlete?region=us&lang=en&contentorigin=espn&isqualified=false&page=1&limit=100&category=offense%3Arushing&sort=rushing.rushingYards%3Adesc&season=2022&seasontype=2"
group = 'rushing'
groupYards = 'rushingYards'
year = 2022

url_flex = "https://site.web.api.espn.com/apis/common/v3/sports/football/nfl/statistics/byathlete?region=us&lang=en&contentorigin=espn&isqualified=false&page=1&limit=100&category=offense%3A{}&sort={}.{}%3Adesc&season={}&seasontype=2".format(group, group, groupYards, year)

# from ESPN's NFL historical data 
def retrieve_historical_stats(position, group, groupYards, year):
    #set request url
    url_flex = "https://site.web.api.espn.com/apis/common/v3/sports/football/nfl/statistics/byathlete?region=us&lang=en&contentorigin=espn&isqualified=false&page=1&limit=100&category=offense%3A{}&sort={}.{}%3Adesc&season={}&seasontype=2".format(group, group, groupYards, year)
    #set headers
    flex_headers = {'Connection': 'keep-alive', 'Accept': 'application/json, text/plain, */*', 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'}
    #grab data
    r_flex = requests.get(url_flex, headers=flex_headers, cookies=espn_cookies) 
    espn_historical_data = r_flex.json()
    # create DF with desired columns
    espn_data_flex = espn_historical_data['athletes']
    position = pd.DataFrame(espn_data_flex)
    return position



In [117]:
def retrieve_historical_stats(group, groupYards, year):
    #set request url
    url_flex = "https://site.web.api.espn.com/apis/common/v3/sports/football/nfl/statistics/byathlete?region=us&lang=en&contentorigin=espn&isqualified=false&page=1&limit=100&category=offense%3A{}&sort={}.{}%3Adesc&season={}&seasontype=2".format(group, group, groupYards, year)
    
    #set headers
    flex_headers = {
        'Connection': 'keep-alive', 
        'Accept': 'application/json, text/plain, */*', 
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'
    }
    
    #grab data
    r_flex = requests.get(url_flex, headers=flex_headers, cookies=espn_cookies) 
    espn_historical_data = r_flex.json()
    
    #Check if the expected data is present
    if 'athletes' in espn_historical_data:
        athletes = espn_historical_data['athletes']
        athletes_df = pd.DataFrame(athletes)
    else: athletes_df = pd.DataFrame()
       
    # this function grabs the id and name of the player from each row under dictionary 'athlete'
    def extract_attributes(athlete_dictionary):
        id_no = athlete_dictionary.get('id')
        name = athlete_dictionary.get('displayName')
        return pd.Series([id_no, name])
    
    # this function extracts individual statistics for each player from the categories list of dicts
    def extract_season_stats(nums):
        rush_att = nums[2].get('totals')[0]
        rush_yd = nums[2].get('totals')[1]
        rush_td = nums[2].get('totals')[5]
        rec = nums[3].get('totals')[0]
        rec_yd = nums[3].get('totals')[1]
        rec_td = nums[3].get('totals')[6]
        pass_att = nums[1].get('totals')[0]
        pass_comp = nums[1].get('totals')[1]
        pass_yd = nums[1].get('totals')[3]
        pass_td = nums[1].get('totals')[7]
        
        return pd.Series([rush_att, rush_yd, rush_td, rec, rec_yd, rec_td, pass_att, pass_comp, pass_yd, pass_td])
       
    # apply both functions across the data
    athletes_df[['id', 'name']] = athletes_df['athlete'].apply(extract_attributes)
    athletes_df[['rush_att', 'rush_yd', 'rush_td', 'rec', 'rec_yd', 'rec_td', 'pass_att', 'pass_comp', 'pass_yd', 'pass_td']] = athletes_df['categories'].apply(extract_season_stats)
    
    return athletes_df[['id', 'name', 'rush_att', 'rush_yd', 'rush_td',
       'rec', 'rec_yd', 'rec_td', 'pass_att', 'pass_comp', 'pass_yd', 'pass_td', 'athlete','categories']]


In [107]:
rbs_df_2022 = retrieve_historical_stats('rushing', 'rushingYards', 2022)

In [114]:
wrs_df_2022 = retrieve_historical_stats('receiving', 'receivingYards', 2022)
wrs_df_2021 = retrieve_historical_stats('receiving', 'receivingYards', 2021)
wrs_df_2020 = retrieve_historical_stats('receiving', 'receivingYards', 2020)

In [116]:
wrs_df_2020.head()

,id,name,rush_att,rush_yd,rush_td,rec,rec_yd,rec_td,pass_att,pass_comp,pass_yd,pass_td,athlete,categories
0,2976212,Stefon Diggs,1,1,0,127,166,20,0,0,0,0,"{'id': '2976212', 'uid': 's:20~l:28~a:2976212'...","[{'name': 'general', 'displayName': 'Own Gener..."
1,15847,Travis Kelce,0,0,0,105,145,23,1,2,4,0,"{'id': '15847', 'uid': 's:20~l:28~a:15847', 'g...","[{'name': 'general', 'displayName': 'Own Gener..."
2,15795,DeAndre Hopkins,1,1,0,115,160,17,0,0,0,0,"{'id': '15795', 'uid': 's:20~l:28~a:15795', 'g...","[{'name': 'general', 'displayName': 'Own Gener..."
3,4262921,Justin Jefferson,1,2,0,88,125,23,0,0,0,0,"{'id': '4262921', 'uid': 's:20~l:28~a:4262921'...","[{'name': 'general', 'displayName': 'Own Gener..."
4,16800,Davante Adams,0,0,0,115,149,18,0,0,0,0,"{'id': '16800', 'uid': 's:20~l:28~a:16800', 'g...","[{'name': 'general', 'displayName': 'Own Gener..."


In [109]:
rbs_df_2022.head()

,id,name,rush_att,rush_yd,rush_td,rec,rec_yd,rec_td,pass_att,pass_comp,pass_yd,pass_td,athlete,categories
0,4047365,Josh Jacobs,340,"1,653",12,53,64,2,0,0,0,0,"{'id': '4047365', 'uid': 's:20~l:28~a:4047365'...","[{'name': 'general', 'displayName': 'Own Gener..."
1,3043078,Derrick Henry,349,"1,538",13,33,41,6,2,2,4,1,"{'id': '3043078', 'uid': 's:20~l:28~a:3043078'...","[{'name': 'general', 'displayName': 'Own Gener..."
2,3128720,Nick Chubb,302,"1,525",12,27,37,1,0,0,0,0,"{'id': '3128720', 'uid': 's:20~l:28~a:3128720'...","[{'name': 'general', 'displayName': 'Own Gener..."
3,3929630,Saquon Barkley,295,"1,312",10,57,76,2,0,0,0,0,"{'id': '3929630', 'uid': 's:20~l:28~a:3929630'...","[{'name': 'general', 'displayName': 'Own Gener..."
4,4045163,Miles Sanders,259,"1,269",11,20,26,0,0,0,0,0,"{'id': '4045163', 'uid': 's:20~l:28~a:4045163'...","[{'name': 'general', 'displayName': 'Own Gener..."


In [55]:
#rbs_df['athlete'][0]

### need to find a way to :
1. join the dfs based on an ID main key
2. create individual time series player-level statistics(for players i want to look at drafting)
3. only include specific positions in the historical dfs, not categories of rushing/passing/receiving bc there is overlap